# 7.2 Arithmetic expressions

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Arithmetic expressions in AMPL are much the same as in other computer languages.
Literal numbers consist of an optional sign preceding a sequence of digits, which may or
may not include a decimal point (for example, -17 or 2.71828 or +.3). At the end of
a literal there may also be an exponent, consisting of the letter d, D, e, or E and an
optional sign followed by digits (1e30 or 7.66439D-07).

Literals, parameters, and variables are combined into expressions by the standard
operations of addition (+), subtraction (-), multiplication (*), division (/), and exponentiation
(ˆ). The familiar conventions of arithmetic apply. Exponentiation has higher
precedence than multiplication and division, which have higher precedence than addition
and subtraction; successive operations of the same precedence group to the left, except
for exponentiation, which groups to the right. Parentheses may be used to change the
order of evaluation.

Arithmetic expressions may also use the div operator, which returns the truncated
quotient when its left operand is divided by its right operand; the mod operator, which
computes the remainder; and the less operator, which returns its left operand minus its
right operand if the result is positive, or zero otherwise. For purposes of precedence and
grouping, AMPL treats div and mod like division, and less like subtraction.

A list of arithmetic operators (and logical operators, to be described shortly) is given
in Table 7-1. As much as possible, AMPL follows common programming languages in its
choice of operator symbols, such as * for multiplication and / for division. There is
sometimes more than one standard, however, as with exponentiation, where some languages
use ˆ while others use \*\*. In this and other cases, AMPL provides alternate
forms. Table 7-1 shows the more common forms to the left, and the alternatives (if any)
<a id=p112></a>
```
Usual                 alternative               type of       type of
style                 style                   operands        result
_____________________________________________________________________
if-then-else                            logical, arithmetic  arithmetic
or                    ||                      logical          logical
exists forall                                 logical          logical
and                   &&                      logical          logical
not (unary)           !                       logical          logical
< <= = <> > >=        < <= == != > >=       arithmetic         logical
in not in                                   object, set        logical
+ - less                                    arithmetic       arithmetic
sum prod min max                            arithmetic       arithmetic
* / div mod                                 arithmetic       arithmetic
+ - (unary)                                 arithmetic       arithmetic
ˆ                     **                    arithmetic       arithmetic

Exponentiation and if-then-else are right-associative; the other operators are
left-associative. The logical operand of if-then-else appears after if, and the
arithmetic operands after then and (optionally) else.
```
**Table 7-1:** Arithmetic and logical operators, in increasing precedence.

to the right; you can mix them as you like, but your models will be easier to read and
understand if you are consistent in your choices.

Another way to build arithmetic expressions is by applying functions to other expressions.
A function reference consists of a name followed by a parenthesized argument or
comma-separated list of arguments; an arithmetic argument can be any arithmetic expression.
Here are a few examples, which compute the minimum, absolute value, and square
root of their arguments, respectively:
```
min(T,20)
abs(sum {i in ORIG} supply[i] - sum {j in DEST} demand[j])
sqrt((tan[j]-tan[k])ˆ2)
```
Table 7-2 lists the built-in arithmetic functions that are typically found in models. Except
for min and max, the names of any of these functions may be redefined, but their original
meanings will become inaccessible. For example, a model may declare a parameter
named tan as in the last example above, but then it cannot also refer to the function
tan.

The set functions card and ord, which were described in Chapter 5, also produce an
arithmetic result. In addition, AMPL provides several "rounding" functions (Section 11.3)
and a variety of random-number functions (Section 7.6 below). A mechanism for
"importing" functions defined by your own programs is described in Appendix A.22.
<a id=p113></a>
```
abs(x)          absolute value, x
acos(x)         inverse cosine, cos − 1 (x)
acosh(x)        inverse hyperbolic cosine, cosh − 1 (x)
asin(x)         inverse sine, sin − 1 (x)
asinh(x)        inverse hyperbolic sine, sinh − 1 (x)
atan(x)         inverse tangent, tan − 1 (x)
atan2(y, x)     inverse tangent, tan − 1 (y / x)
atanh(x)        inverse hyperbolic tangent, tanh − 1 (x)
cos(x)          cosine
cosh(x)         hyperbolic cosine
exp(x)          exponential, e x
log(x)          natural logarithm, log e (x)
log10(x)        common logarithm, log 10 (x)
max(x, y, ...)  maximum (2 or more arguments)
min(x, y, ...)  minimum (2 or more arguments)
sin(x)          sine
sinh(x)         hyperbolic sine
sqrt(x)         square root
tan(x)          tangent
tanh(x)         hyperbolic tangent
```
**Table 7-2:** Built-in arithmetic functions for use in models.

Finally, the indexed operators such as Σ and Π from algebraic notation are generalized
in AMPL by expressions for iterating operations over sets. In particular, most large-scale
linear programming models contain iterated summations:
```
sum {i in ORIG} supply[i]
```
The keyword sum may be followed by any indexing expression. The subsequent arithmetic
expression is evaluated once for each member of the index set, and all the resulting
values are added. Thus the sum above, from the transportation model of Figure 3-1a, represents
the total supply available, at all origins. The sum operator has lower precedence
than \*, so the objective of the same model can be written
```
sum {i in ORIG, j in DEST} cost[i,j] * Trans[i,j]
```
to represent the total of cost[i,j] * Trans[i,j] over all combinations of origins
and destinations. The precedence of sum is higher than that of + or -, however, so for
the objective of the multiperiod production model in Figure 6-3 we must write
```
sum {p in PROD, t in 1..T}
 (sum {a in AREA[p]} revenue[p,a,t]*Sell[p,a,t] -
    prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
```
The outer sum applies to the entire parenthesized expression following it, while the inner
sum applies only to the term revenue[p,a,t] * Sell[p,a,t].
<a id=p114></a>

Other iterated arithmetic operators are prod for multiplication, min for minimum,
and max for maximum. As an example, we could use
```
max {i in ORIG} supply[i]
```
to describe the greatest supply available at any origin.

Bear in mind that, while an AMPL arithmetic function or operator may be applied to
variables as well as to parameters or to numeric members of sets, most operations on variables
are not linear. AMPL's requirements for arithmetic expressions in a linear program
are described in Section 8.2. Some of the nonlinear functions of variables that can be
handled by certain solvers are discussed in Chapter 18.
